In [1]:
# TargetEncoder
#!pip install -q scikit-learn==1.4
# KerasClassifier
#!pip install -q --no-deps scikeras

In [2]:
#현재 커널에 설치가 안되어서 삽질..
#import sys
#import subprocess
# 현재 Python 실행 파일 경로를 사용하여 pip 설치 명령 실행
#subprocess.check_call([sys.executable, "-m", "pip", "install", "scikeras"])


In [1]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from scikeras.wrappers import KerasClassifier

2024-07-24 19:09:09.919851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Read and split the data
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train_origin.sample(frac=0.01, random_state = 42)

In [5]:
# 범주형 변수를 팩터로 변환 (카테고리형)
train.iloc[:,[1,3,4,5,6,7,9]] = train.iloc[:,[1,3,4,5,6,7,9]].astype('category')

# 최소-최대 정규화 (Min-Max 스케일링)
scaler = MinMaxScaler()
train.iloc[:,[2,8,10]] = scaler.fit_transform(train.iloc[:,[2,8,10]])

# 이분변수 생성: "Annual_Premium" == 2630.0 인 경우
train['Annual_Premium_Binary'] = (train['Annual_Premium'] == 2630.0).astype('category')

# 로그 변환된 "Annual_Premium" 변수 생성
train['Annual_Premium_Log'] = np.where(train['Annual_Premium'] > 0, np.log1p(train['Annual_Premium']), 0)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id', 'Annual_Premium'])

# 원-핫 인코딩 (One-Hot Encoding)
category_columns = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Annual_Premium_Binary']
train = pd.get_dummies(train, columns=category_columns, drop_first=True, dtype=int)

# XGBoost에서 발생하는 문제 해결
train.columns = train.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')

# 특징과 레이블 분리
X = train.drop(columns=['Response'])
y = train['Response']

# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# The Keras model with two hidden layers
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(1, kernel_initializer='lecun_normal', activation='sigmoid'))
    return model

keras_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=0.003),
    validation_split=0.05,
    batch_size=1024,
    validation_batch_size=65536,
    epochs=20,
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3), keras.callbacks.EarlyStopping(patience=5)]
)

In [7]:
# Execute the pipeline and measure the auc score
keras_model.fit(X_train, y_train)

Epoch 1/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.3473 - val_loss: 0.2661 - learning_rate: 0.0030
Epoch 2/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2670 - val_loss: 0.2680 - learning_rate: 0.0030
Epoch 3/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2692 - val_loss: 0.2667 - learning_rate: 0.0030
Epoch 4/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2684 - val_loss: 0.2672 - learning_rate: 0.0030
Epoch 5/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2674 - val_loss: 0.2644 - learning_rate: 3.0000e-04
Epoch 6/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2664 - val_loss: 0.2643 - learning_rate: 3.0000e-04
Epoch 7/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2658 - val_loss: 0.2642 - learning_rate: 3.0000e-04
Epoch 8/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2673 - val_loss: 0.2640 - learning_rate: 3.0000e-04
Epoch 9/20
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2675 - val_loss: 0.2641 - learning_rate: 3.0000e-04
Epoch 1

KerasClassifier(
	model=<function get_model at 0x13c735e40>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.src.optimizers.adamw.AdamW object at 0x13c7131a0>
	loss=binary_crossentropy
	metrics=None
	batch_size=1024
	validation_batch_size=65536
	verbose=1
	callbacks=[<keras.src.callbacks.reduce_lr_on_plateau.ReduceLROnPlateau object at 0x13c2ef0e0>, <keras.src.callbacks.early_stopping.EarlyStopping object at 0x13c713da0>]
	validation_split=0.05
	shuffle=True
	run_eagerly=False
	epochs=20
	class_weight=None
)

In [8]:
y_pred = keras_model.predict_proba(X_valid)[:, 1]
print(f"# AUC: {roc_auc_score(y_valid, y_pred):.5f}")


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
# AUC: 0.85160


랜덤서치를 통한 하이퍼파라미터 최적화

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

# Define the model function
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(1, kernel_initializer='lecun_normal', activation='sigmoid'))
    return model

# Wrap the Keras model with KerasClassifier
keras_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW,
    validation_split=0.05,
    batch_size=1024,
    validation_batch_size=65536,
    epochs=20,
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3), keras.callbacks.EarlyStopping(patience=5)]
)

# Define the hyperparameter space
param_distributions = {
    'optimizer__learning_rate': loguniform(1e-5, 1e-2),
    'batch_size': [64, 128, 256, 512, 1024, 2048],
    'epochs': [10, 20, 30]
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=keras_model,
    param_distributions=param_distributions,
    n_iter=10,  # number of parameter settings that are sampled
    cv=3,  # number of cross-validation folds
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='roc_auc'
)

# Execute the search
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", random_search.best_params_)

# Evaluate the model
best_model = random_search.best_estimator_
y_pred = best_model.predict_proba(X_valid)[:, 1]
print(f"# AUC: {roc_auc_score(y_valid, y_pred):.5f}")
print(random_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


2024-07-24 11:16:39.812106: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 11:16:39.818735: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 11:16:39.831408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 11:16:39.833420: I tensorflow/core/platform/cpu_featu

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 0.8294 - val_loss: 0.6931 - learning_rate: 2.9380e-05
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 0.6500 - val_loss: 0.4669 - learning_rate: 3.5506e-05
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - loss: 0.5951 - val_loss: 0.4344 - learning_rate: 3.5506e-05
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.6148 - val_loss: 0.4721 - learning_rate: 3.5506e-05
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.4359 - val_loss: 0.3943 - learning_rate: 3.5506e-05
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.4080 - val_loss: 0.3644 - learning_rate: 3.5506e-05
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.6524 - val_loss: 0.5635 - learning_rate: 2.9380e-05
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.4350 - val_loss: 0.3793 - learning_rate: 3.5506e-05
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/ste

In [15]:
# the best hyperparameters from the random search
random_search.best_params_

{'batch_size': 128,
 'epochs': 30,
 'optimizer__learning_rate': 0.00023345864076016249}

In [12]:
# 데이터 샘플링
train = train_origin.sample(frac=0.01, random_state = 42)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id'])

# 범주형 변수(2~3개 클래스) 인코딩
def encoding(train):
    gender_mapping = {'Male': 0, 'Female': 1}
    vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
    vehicle_damage_mapping = {'No': 0, 'Yes': 1}

    train['Gender'] = train['Gender'].map(gender_mapping)
    train['Vehicle_Age'] = train['Vehicle_Age'].map(vehicle_age_mapping)
    train['Vehicle_Damage'] = train['Vehicle_Damage'].map(vehicle_damage_mapping)

    return train

train = encoding(train)

'''(같은 건데 왠지 위 방식이 미세하게 잘 나온다)
cat_columns_simple = ['Gender','Vehicle_Age','Vehicle_Damage']
train[cat_columns_simple] = train[cat_columns_simple].astype('category')
train['Gender'] = train['Gender'].cat.codes
train['Vehicle_Age'] = train['Vehicle_Age'].cat.codes
train['Vehicle_Damage'] = train['Vehicle_Damage'].cat.codes
'''

#Previously_Insured 교차항
'''
train['Previously_Insured_Annual_Premium'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Annual_Premium'].astype(str))[0]
train['Previously_Insured_Vehicle_Age'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vehicle_Age'].astype(str))[0]
train['Previously_Insured_Vehicle_Damage'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vehicle_Damage'].astype(str))[0]
train['Previously_Insured_Vintage'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vintage'].astype(str))[0]
'''

#Age 교차항
train['Age_bins'] = pd.cut(train['Age'], bins=7).cat.codes
train['Age_x_Vehicle_Age'] = train['Age_bins'] * train['Vehicle_Age']
train['Age_x_Vehicle_Damage'] = train['Age_bins'] * train['Vehicle_Damage']
train['Age_x_Previously_Insured'] = train['Age_bins'] * train['Previously_Insured']

# 범주형 변수 타겟 인코딩
cat_columns = ['Region_Code', 'Policy_Sales_Channel', 'Vintage']
train.loc[:,cat_columns] = train.loc[:,cat_columns].astype('category')

target_encoder = TargetEncoder()
train[cat_columns] = target_encoder.fit_transform(train[cat_columns],train['Response'])

#수치형 변수 + 타겟 인코딩 변수 표준화
scaler = MinMaxScaler()
num_columns = ['Age', 'Annual_Premium','Region_Code', 'Policy_Sales_Channel', 'Vintage']
train[num_columns] = scaler.fit_transform(train[num_columns])

# 예측변수 분리 및 train, valid set 분리
X = train.drop(['Response'], axis=1)
y = train['Response']

# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Recreate the model with the best hyperparameters
keras_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=0.00023),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=30,  # Increase the number of epochs for further training
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3), keras.callbacks.EarlyStopping(patience=5)]
)

# Train the model further
keras_model.fit(X_train, y_train)

# Evaluate the further trained model
y_pred_further = keras_model.predict_proba(X_valid)[:, 1]
print(f"# Further Trained AUC: {roc_auc_score(y_valid, y_pred_further):.5f}")

Epoch 1/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.3085 - val_loss: 0.2641 - learning_rate: 2.3000e-04
Epoch 2/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2634 - val_loss: 0.2606 - learning_rate: 2.3000e-04
Epoch 3/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2659 - val_loss: 0.2559 - learning_rate: 2.3000e-04
Epoch 4/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2643 - val_loss: 0.2560 - learning_rate: 2.3000e-04
Epoch 5/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2628 - val_loss: 0.2559 - learning_rate: 2.3000e-04
Epoch 6/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2633 - val_loss: 0.2555 - learning_rate: 2.3000e-04
Epoch 7/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.2618 - val_loss: 0.2565 - learning_rate: 2.3000e-04
Epoch 8/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2644 - val_loss: 0.2599 - learning_rate: 2.3000e-04
Epoch 9/30
684/684 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2657 - val_loss: 0.2569 - 

In [34]:
# 모델 정의 함수
def get_model2(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    
    return model

# 학습률 스케줄러 함수 정의
def lr_scheduler(epoch, lr):
    decay_rate = 0.96
    decay_step = 10
    new_lr = lr * (decay_rate ** (epoch // decay_step))
    print(f"Epoch {epoch}: Learning rate is {new_lr}")
    return new_lr

# LearningRateScheduler 콜백 설정
lr_schedule = LearningRateScheduler(lr_scheduler)

# Recreate the model with the best hyperparameters
keras_model = KerasClassifier(
    get_model2,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=2e-03),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=100,  # Increase the number of epochs for further training
    callbacks=[lr_schedule, keras.callbacks.EarlyStopping(patience=20)]
)

# Train the model further
keras_model.fit(X_train, y_train)

# Evaluate the further trained model
y_pred_further = keras_model.predict_proba(X_valid)[:, 1]
print(f"# Further Trained AUC: {roc_auc_score(y_valid, y_pred_further):.5f}")

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2855 - val_loss: 0.2584 - learning_rate: 0.0020
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2652 - val_loss: 0.2576 - learning_rate: 0.0020
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2653 - val_loss: 0.2549 - learning_rate: 0.0020
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2615 - val_loss: 0.2600 - learning_rate: 0.0020
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2639 - val_loss: 0.2564 - learning_rate: 0.0020
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2614 - val_loss: 0.2563 - learning_rate: 0.0020
Epoch 6: Learning rate is 0.00200000009499490